# Bing API Search Tool

## Import dependencies

In [1]:
# import the necessary packages
from requests import exceptions
import requests
import cv2
import os
import json

## Define variables and take input
-  API key and URL are defined
-  Input is stored for directory creation and search query
-  Result limit and group is defined
-  Exceptions are defined for handling later

In [2]:
#Ask for search term
print("What dog breed did you want to find pictures for?")

#Set search term and directory for storage
searchTerm = input()
directoryName = "dataset/" + searchTerm

#Print to check
print("------------------------------------------")
print("You're searching for: " + searchTerm)
print("The images will be saved to: " + directoryName)
print("------------------------------------------")

#Define API key, URL and result limits
url = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"
key = "3b61755b6e6149f6b8b52a61cee5d7e6"
maxResults = 250
groupSize = 50

#Exception handling block
EXCEPTIONS = set([IOError, FileNotFoundError,
	exceptions.RequestException, exceptions.HTTPError,
	exceptions.ConnectionError, exceptions.Timeout])

What dog breed did you want to find pictures for?
chihuahua
------------------------------------------
You're searching for: chihuahua
The images will be saved to: dataset/chihuahua
------------------------------------------


## API Specific parameters
Web Search API - v7
The Search API provides a similar (but not exact) experience to Bing.com/Search by returning search results that Bing determines are relevant to the specified query. The results also identify the order that you must display the content in (see Using Ranking to Display Results). The response may also include related search links and suggest a query string that may more accurately represent the user's intent. Typically, you will call this API instead of calling the other APIs in the Bing API family, such as the Image API or News API.

Request URL
https://api.cognitive.microsoft.com/bing/v7.0/search[?q][&count][&offset][&mkt][&safesearch]

__Request parameters__
-  q string : The user's search query string
-  count (optional) _number_ : The number of search results to return in the response. The actual number delivered may be less than requested.
-  offset (optional) _number_ : The zero-based offset that indicates the number of search results to skip before returning results.
-  mkt (optional) _string_ : The market where the results come from. Typically, this is the country where the user is making the request from; however, it could be a different country if the user is not located in a country where Bing delivers results. The market must be in the form -. For example, en-US.
-  A filter used to filter results for adult content.

__Request headers__
-  Ocp-Apim-Subscription-Key _string_ : Subscription key which provides access to this API. Found in your Cognitive Services accounts.

In [3]:
#Set bing search parameters
#q (string) : The user's search query string
#count (optional) : The number of search results to return in the response. The actual number delivered may be less than requested.
#offset (optional) : The zero-based offset that indicates the number of search results to skip before returning results.

requestParams = {"q": searchTerm,
                 "offset": 0,
                 "count": groupSize}

# Set bing search header
# Ocp-Apim-Subscription-Key (string) : Subscription key which provides access to this API. Found in your Cognitive Services accounts.

requestHeader = {"Ocp-Apim-Subscription-Key" : key}

# Start the searching block

print("--- Searching for your query")
search = requests.get(url, headers=requestHeader, params=requestParams)

#Checks if the request was a success and performs dump of the json
search.raise_for_status()
searchResults = search.json()
print(json.dumps(searchResults, indent=4))

--- Searching for your query
{
    "_type": "Images",
    "instrumentation": {
        "_type": "ResponseInstrumentation"
    },
    "readLink": "https://api.cognitive.microsoft.com/api/v7/images/search?q=chihuahua",
    "webSearchUrl": "https://www.bing.com/images/search?q=chihuahua&FORM=OIIARP",
    "queryContext": {
        "originalQuery": "chihuahua",
        "alterationDisplayQuery": "chihuahua",
        "alterationOverrideQuery": "+chihuahua",
        "alterationMethod": "AM_JustChangeIt",
        "alterationType": "CombinedAlterationsChained"
    },
    "totalEstimatedMatches": 730,
    "nextOffset": 58,
    "value": [
        {
            "webSearchUrl": "https://www.bing.com/images/search?view=detailv2&FORM=OIIRPO&q=chihuahua&id=295489CB11CDADA60A63415A36818DB63974DCD3&simid=607991359928927097",
            "name": "Chihuahua Breed Guide - Learn about the Chihuahua.",
            "thumbnailUrl": "https://tse2.mm.bing.net/th?id=OIP.q0OwMiPTwIKnJa_F7DZ3YQHaFj&pid=Api",
       

## Set upper limit to results and initialize loop counter
-  Use min to set the max limit to match with the search results (rememdies situations with low result)

In [4]:
#Create image counter for loop and limit the results to 100
imageTotal = 0
numResults = min(searchResults["totalEstimatedMatches"], maxResults)

## For loop to go through the results and save the pictures
1. Loop through each image starting with offset 0
2. Use request.get with API url, headers and params to find json data
3. Store JSON into results
4. Iterate through all entries in results
5. Try/Except block referencing exceptions from earlier to prevent program from breaking due to errors
6. Create naming convention for pictures based on file extension, padded 0s

In [5]:
# loop over the estimated number of results in `groupSize` groups
for i in range(0, numResults, groupSize):
	print("--- Requesting image {}-{} of {} ---".format(i, i + groupSize, numResults))
	requestParams["offset"] = i
	search = requests.get(url, headers=requestHeader, params=requestParams)
	search.raise_for_status()
	results = search.json()
	print("--- Storing image {}-{} of {} ---".format(i, i + groupSize, numResults))

	# loop over the results
	for result in results["value"]:
		# Try/Except block for downloads
		try:
			#Try to download
			print("Querying: " + result["contentUrl"])
			r = requests.get(result["contentUrl"], timeout=30)

			#Create a filepath using the actual file extension found from the rfind command
            #Name the file using imageTotal counter and padded 0s
			fileExtension = result["contentUrl"][result["contentUrl"].rfind("."):]
			outputImage = os.path.sep.join(
                [directoryName,
                "{}{}"
                .format(str(imageTotal)
                .zfill(8), fileExtension)])

			#Download and save the file
			imageFile = open(outputImage, "wb")
			imageFile.write(r.content)
			imageFile.close()
            
		except Exception as e:
			#Check exception and print the conflict url as well as the error code
			if type(e) in EXCEPTIONS:
				print("Skipping " + result["contentUrl"] + " Error code: " + str(e))
				continue

		#Use open CV to load the image
		image = cv2.imread(outputImage)
		
        #Check to see if the image actually exists, delete if empty
		if image is None:
			print("Deleting: " + outputImage)
			os.remove(outputImage)
			continue

		#Increment loop counter
		imageTotal += 1

--- Requesting image 0-50 of 250 ---
--- Storing image 0-50 of 250 ---
Querying: http://www.petpaw.com.au/wp-content/uploads/2014/04/Chihuahua-2.jpg
Querying: http://www.petpaw.com.au/wp-content/uploads/2014/04/Chihuahua-4.jpg
Querying: http://www.aboutdogfacts.com/wp-content/uploads/2015/06/white-apple-head-chihuahua.jpg
Querying: http://www.petguide.com/wp-content/uploads/2013/02/chihuahua-21.jpg
Skipping http://www.petguide.com/wp-content/uploads/2013/02/chihuahua-21.jpg Error code: HTTPConnectionPool(host='127.0.0.1', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002CF7E9E2BE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))
Querying: https://media.buzzle.com/media/images-en/photos/mammals/dogs/1200-22584015-apple-head-chihuahua.jpg
Querying: http://deceth.com/wp-content/uploads/2015/07/chihuahua-bling.jpg
Querying: htt

Querying: http://media.buzzle.com/media/images-en/gallery/mammals/dogs/1200-459901463-chihuahua-dog.jpg
Querying: http://www.dog-learn.com/dog-breeds/chihuahua/images/chihuahua.jpg
Querying: http://www.golfian.com/wp-content/uploads/2016/07/Confused-Black-Chihuahua-Dog-Face-Picture.jpg
Querying: http://images2.fanpop.com/image/photos/9400000/Chihuahua-chihuahua-dog-lovers-9414647-1360-2048.jpg
Querying: http://hairstylecamp.com/wp-content/uploads/Long-Haired-Chihuahua-care-tips.jpg
Querying: http://doglers.com/wp-content/uploads/2016/09/Chihuahua-Puppies-6.jpg
Querying: https://www.pets4homes.co.uk/images/articles/1313/large/why-do-chihuahuas-and-other-small-dogs-shiver-52ea36662ee34.jpg
Querying: https://www.pets4homes.co.uk/images/breeds/41/large/70ae8795b2ed020f8fc570be2bb8c95a.JPG
Querying: http://www.golfian.com/wp-content/uploads/2016/07/Small-Black-and-Brown-Chihuahua-Dog-Sitting-On-Bed.jpg
Querying: https://pixfeeds.com/images/dogs/1280-90165199-chihuahua-puppies.jpg
Querying: 

Querying: https://i.pinimg.com/736x/41/88/83/418883ed19832bbf7fe76ab5bba08d1c--long-haired-chihuahua-puppies-long-coat-chihuahua.jpg
Querying: http://images4.fanpop.com/image/photos/21300000/chihuahua-chihuahuas-21370882-1280-960.jpg
Querying: https://pixfeeds.com/images/dogs/1280-121347148-puppy-chihuahua.jpg
Querying: https://media.buzzle.com/media/images-en/photos/mammals/dogs/1200-84655873-cute-chihuahua.jpg
Querying: http://www.chiwawadogs.net/wp-content/uploads/2013/06/chipuppy-e1380226702148.jpg
Querying: https://s-media-cache-ak0.pinimg.com/736x/8e/08/90/8e0890844950e21387fcf4ffbe938167.jpg
Querying: https://i.pinimg.com/736x/54/af/30/54af3005e93a91ba418eaf21596c6fff--long-hair-chihuahua-chihuahua-dogs.jpg
Querying: https://www.101dogbreeds.com/wp-content/uploads/2015/10/Italian-Greyhuahua-Dog.jpg
Querying: https://s-media-cache-ak0.pinimg.com/originals/35/4c/4f/354c4fecd3ec6884cf159d8c214d514a.jpg
Querying: http://wallpapersdsc.net/wp-content/uploads/2016/10/Chihuahua-Widescre

Querying: https://i.pinimg.com/736x/7b/4f/78/7b4f78b7ca6dd58a14487c466e30cba3--chihuahua-puppies-for-sale-chihuahua-pups.jpg
Querying: http://2.bp.blogspot.com/-mxm91yNBa5Y/Te3sdaztvZI/AAAAAAAAHTs/w8oMXFCNyzI/s1600/chihuahua-dog3.jpg
Querying: http://4.bp.blogspot.com/-3jqbd0dXDMA/UVLz1GEaAnI/AAAAAAAAmxo/W9LYihsPYbg/s1600/beverly_hills_chihuahua02.jpg
Querying: http://hdbackgroundspic.com/wp-content/uploads/2017/06/beautiful-white-chihuahua.jpg
Querying: http://images4.fanpop.com/image/photos/16000000/Lovely-Chihuahua-chihuahuas-16093415-1600-1200.jpg
Querying: https://media.buzzle.com/media/images-en/photos/mammals/dogs/1200-19186756-chihuahua-yorkshire-terrier-hybrid.jpg
Querying: https://www.pets4homes.co.uk/images/articles/1746/large/chihuahua-dogs-and-health-conditions-53a006ee19a1f.jpg
Querying: http://www.perrosamigos.com/Uploads/perrosamigos.com/ImagenesGrandes/chihuahua-0-0.jpg
Querying: http://lnx.dolcechihuahua.it/wp-content/uploads/2014/02/DSCN2337.jpg
Querying: https://ckc